## Types of Chat Bot's
In the world of machine learning and AI there are many different kinds of chat bots. Some chat bots are virtual assistants, others are just there to talk to, some are customer support agents and you've probably seen some of the ones used by businesses to answer questions. For this tutorial we will be creating a relatively simple chat bot that will be be used to answer frequently asked questions.

## Install Packages
Before starting to work on our chatbot we need to download a few python packages. Please note as of writing this these packages will ONLY WORK IN PYTHON 3.6. Hopefully this will be fixed in the future.

We will simply use pip to install the following:
- numpy
- nltk
- tensorflow
- tflearn

Simply go to CMD and type: pip install "package name". Where you will replace "package_name" with all of the entries listed above.

### Training Data
Now it's time to understand what kind of data we will need to provide our chatbot with. Since this is a simple chatbot we don't need to download any massive datasets. We will just use data that we write ourselves. To follow along with the tutorial properly you will need to create a .JSON file that contains the same format as the one seen below. I've called my file "intents.json".

In [1]:
#from google.colab import drive
#drive.mount("/content/drive")

In [2]:
!ls '/content/drive/MyDrive/Colab Notebooks/notebooks'

ls: /content/drive/MyDrive/Colab Notebooks/notebooks: No such file or directory


What we are doing with the JSON file is creating a bunch of messages that the user is likely to type in and mapping them to a group of appropriate responses. The tag on each dictionary in the file indicates the group that each message belongs too. With this data we will train a neural network to take a sentence of words and classify it as one of the tags in our file. Then we can simply take a response from those groups and display that to the user. The more tags, responses, and patterns you provide to the chatbot the better and more complex it will be.

### Loading our JSON Data
We will start by importing some modules and loading in our json data. Make sure that your .json file is in the same directory as your python script!

In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow as tf
import random

import json
with open('intents.json') as file:
    data = json.load(file)
    
print('done')

Instructions for updating:
non-resource variables are not supported in the long term
done


In [4]:
#!pip install tflearn

In [5]:
import nltk 
#nltk.download('punkt')

## Extracting Data
Now its time to take out the data we want from our JSON file. We need all of the patterns and which class/tag they belong to. We also want a list of all of the unique words in our patterns (we will talk about why later), so lets setup some blank lists to store these values.

words = []
labels = []
docs_x = []
docs_y = []
Now its time to loop through our JSON data and extract the data we want. For each pattern we will turn it into a list of words using nltk.word_tokenizer, rather than having them as strings. We will then add each pattern into our docs_x list and its associated tag into the docs_y list.

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
In the next tutorial we will do some preprocessing of this data and get it ready to feed to our neural network.

In [6]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
print(labels)

['greeting', 'goodbye', 'what', 'USCIS', 'DACA', 'DACA_DETAIL', 'hours']


## Word Stemming
You may have heard me talk about word stemming in the previous tutorial. Stemming a word is attempting to find the root of the word. For example, the word "thats" stem might be "that" and the word "happening" would have the stem of "happen". We will use this process of stemming words to reduce the vocabulary of our model and attempt to find the more general meaning behind sentences.

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

This code will simply create a unique list of stemmed words to use in the next step of our data preprocessing.

## Bag of Words
Now that we have loaded in our data and created a stemmed vocabulary it's time to talk about a bag of words. As we know neural networks and machine learning algorithms require numerical input. So out list of strings wont cut it. We need some way to represent our sentences with numbers and this is where a bag of words comes in. What we are going to do is represent each sentence with a list the length of the amount of words in our models vocabulary. Each position in the list will represent a word from our vocabulary. If the position in the list is a 1 then that will mean that the word exists in our sentence, if it is a 0 then the word is nor present. We call this a bag of words because the order in which the words appear in the sentence is lost, we only know the presence of words in our models vocabulary.

As well as formatting our input we need to format our output to make sense to the neural network. Similarly to a bag of words we will create output lists which are the length of the amount of labels/tags we have in our dataset. Each position in the list will represent one distinct label/tag, a 1 in any of those positions will show which label/tag is represented.

In [7]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)
print(output)

[[0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0]]


## Developing a Model
Now that we have preprocessed all of our data we are ready to start creating and training a model. For our purposes we will use a fairly standard feed-forward neural network with two hidden layers. The goal of our network will be to look at a bag of words and give a class that they belong too (one of our tags from the JSON file).

We will start by defining the architecture of our model. Keep in mind that you can mess with some of the numbers here and try to make an even better model! A lot of machine learning is trial an error.


## Training & Saving the Model
Now that we have setup our model its time to train it on our data! To do these we will fit our data to the model. The number of epochs we set is the amount of times that the model will see the same information while training.

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")
Once we are done training the model we can save it to the file model.tflearn for use in other scripts.

In [8]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model_chat101/model.tflearn")

Training Step: 3999  | total loss: 0.01236 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.01236 - acc: 1.0000 -- iter: 24/30
Training Step: 4000  | total loss: 0.01252 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.01252 - acc: 1.0000 -- iter: 30/30
--
INFO:tensorflow:/Users/dacarpen/git/CI5-Chat-in-the-Hat/statTracker/model_chat101/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


## Making Predictions
Now its time to actually use the model! Ideally we want to generate a response to any sentence the user types in. To do this we need to remember that our model does not take string input, it takes a bag of words. We also need to realize that our model does not spit out sentences, it generates a list of probabilities for all of our classes. This makes the process to generate a response look like the following:
– Get some input from the user
– Convert it to a bag of words
– Get a prediction from the model
– Find the most probable class
– Pick a response from that class



The bag_of_words function will transform our string input to a bag of words using our created words list. The chat function will handle getting a prediction from the model and grabbing an appropriate response from our JSON file of responses.

Now run the program and enjoy chatting with your bot!

In [10]:
import requests
url="http://localhost:5000/record"

In [11]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

def chat():
    print(counts)
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        results_score = numpy.amax(results)
        tag = labels[results_index]
        #record the result for analysis
        recordData = {'intent':tag, 'count':1, 'score': "{:.4f}".format(results_score), 'input': inp}
        x = requests.post(url, json = recordData )
        # print(x.text)   
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))
        
def question(text):
    results = model.predict([bag_of_words(text, words)])
    results_index = numpy.argmax(results)
    results_score = numpy.amax(results)
    tag = labels[results_index]
    recordData = {'intent':tag, 'count':1, 'score': "{:.4f}".format(results_score), 'input': text}
    x = requests.post(url, json = recordData )
    for tg in data["intents"]:
        if tg['tag'] == tag:
            responses = tg['responses']

    result = (random.choice(responses))
    return result

# run this to chat 
chat()

In [12]:
#run this to send individual data 
question('hello?')

'Hello!'

In [13]:
import json
  
# Opening JSON file
f = open('intents.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
for i in data['intents']:
    print(i)
  
# Closing file
f.close()

{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}
{'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}
{'tag': 'what', 'patterns': ['what are you', 'who are you', 'what do you do', 'what is your purpose', 'what is this'], 'responses': ['I am USCIS chat bot for handeling questions and customer service', 'I am a full AI chat bot for USCIS'], 'context_set': ''}
{'tag': 'USCIS', 'patterns': ['what is USCIS', 'USCIS dose what?', 'whats USCIS do?'], 'responses': ["U.S. Citizenship and Immigration Services is an agency of the United States Department of Homeland Security that administers the country's naturalization and immigration system"], 'context_set': ''}
{'tag': 'DACA', 'patterns': ['What is pu

In [14]:
questions =[]
answers = []

for i in data['intents']:
    questions.append(i['patterns'])
    answers.append(i['responses'][0])
    
print(questions)
print('')
print('')
print(answers)

[['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], ['what are you', 'who are you', 'what do you do', 'what is your purpose', 'what is this'], ['what is USCIS', 'USCIS dose what?', 'whats USCIS do?'], ['What is purpose of DACA?', 'What is deferred action?', 'What is DACA?'], ['Why is DACA ending?', 'Why is DACA phasing out', 'How can I get more DACA information?', 'Frequently asked DACA questions'], ['when are you open', 'what are your hours', 'hours of operation', 'what is your availability']]


['Hello!', 'Sad to see you go :(', 'I am USCIS chat bot for handeling questions and customer service', "U.S. Citizenship and Immigration Services is an agency of the United States Department of Homeland Security that administers the country's naturalization and immigration system", 'Under existing regulations, an individual whose case has been deferred is eligible to receive employment authorizatio

In [15]:
# set up parrot to do parra phrases 
# Import libraries
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")
print('parrot loaded ready to work')


parrot loaded ready to work


In [16]:
##Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

In [17]:
#build compresed method 
def get_para(phrase, num, length):
    texts = []
    para_phrases = parrot.augment(input_phrase=phrase, use_gpu=False, diversity_ranker='levenshtein', do_diverse=False,
                                max_return_phrases=num, max_length=length, adequacy_threshold=0.9, fluency_threshold=0.9)
    for para_phrase in para_phrases:
        text = para_phrase[0]
        texts.append(text)
    
    return texts
        

text = 'Whats the most delicious papayas?'
texts = get_para(text, 10, len(text)+5)
print(texts)

['list the most delicious papayas you have ever eaten?', 'tell me the most delicious papaya you have ever had?', 'tell me the best and sexiest papaya dish?', "what's the most delicious papaya you've ever had?", 'show some of the best papayas?', 'which is the most delicious papaya?', "what's the most delicious papaya?", 'what is the most delicious papaya?']


In [18]:
# now run all intents and make 0 succsess 1 fail
results = []

index =0
for x in questions:
    answer = answers[index]
    for i in x:
        texts = get_para(i, 10, len(text)+5)
        texts.append(i)
        print(texts)
        for text in texts:
            res = question(text)
            #print('AI :', res)
            #print('answer :', answer)
            if(answer == res):
                results.append(0)
            else:
                results.append(1)
    index +=1
    print(results)

print('')
print('these are the final results')
print(results)

['Hi', 'Hi']
["what's your general state?", "what's up?", 'how are you doing?', 'how are you?', 'How are you']
['who is there?', 'is there anybody there?', 'is anyone here?', 'is anyone there?', 'Is anyone there?']
['Hello', 'Hello']
['happy day', 'good day', 'Good day']
["what's going on?", 'what happened?', 'what was wrong?', "what's up?", "what's up", 'Whats up']
[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0]
['cya', 'cya']
['see you later', "we'll see you later", "maybe i'll see you later", 'See you later']
['sorry', 'goodbye', 'Goodbye']
["i'm going to leave", "i'm leaving", 'I am Leaving']
['have a good day', 'Have a Good day']
[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1]
['what are you doing?', 'what are you like?', 'what are you?', 'what are you']
['who are you?', 'who are you']
["what's the thing you're doing?", 'what can you do?', 'what do you do?', 'what do you do']
["what's your goal?"

In [21]:
# now run new questions
newQuestions = ["How many roads must a man walk down before he's considered a man", "Where can I find Form-158-b", "Do I need to include a hiring site for remote work locations",
               "What is a NAICS code?", "am I exempt?", "can I start the employee working before E-verify returns a final asnwer"]

for i in newQuestions:
    texts = get_para(i, 10, len(text)+5)
    texts.append(i)
    print(texts)
    for text in texts:
        res = question(text)

['how many streets must a man walk before he is considered to be a man?', 'how many streets must a man walk before he can be considered a man?', "How many roads must a man walk down before he's considered a man"]
['where can i find the form 158 b?', 'where can i find the form 158-b?', 'where can i find a form 158-b?', 'Where can I find Form-158-b']
['do i need to include a location for remote work sites?', 'will i need to include a site for remote work sites?', 'do i need to include a hiring site for remote work locations?', 'Do I need to include a hiring site for remote work locations']
["what's naics code?", "what's an naics code?", 'what is naics code?', 'what is a naics code?', 'What is a NAICS code?']
['is my tax exemption applied to my case?', 'are there exceptions?', 'am i exempt?', 'are i exempt?', 'am I exempt?']
['can I start the employee working before E-verify returns a final asnwer', 'can I start the employee working before E-verify returns a final asnwer']
